In [1]:
import numpy as np
import plotly.express as px
import pandas as pd
import serial
import time
from enum import Enum
import json
from radar_configuration import Radar
from threading import Thread

stepAngle = 0.45

scanningLowerStepSize = 1
scanningUpperStepSize = 1

maxAngleUpper = 45
maxStepsOfUpper = maxAngleUpper/(stepAngle*scanningUpperStepSize)

maxAngleLower = 45
maxStepsOfLower = maxAngleLower/(stepAngle*scanningLowerStepSize)

calibrateLowerStepSize = 10
calibrateLowerTotalStepsCount = 20



configuration_file = open('configuration.json',)
configuration_json = json.load(configuration_file)
arduino_port = configuration_json["ARDUINO_PORT"]


class Motors(Enum):
    LOWER = 'l'
    UPPER = 'u'


class Direction(Enum):
    POSITIVE = 1
    NEGATIVE = -1



radar = Radar()
global_distance = -1
def get_readings_thread():
    global global_distance
    while(True):
        global_distance = radar.get_median_distance(3)
        print("global distance = ",global_distance)


def set_up():
    arduino = serial.Serial()
    arduino.baudrate = 9600
    arduino.port = arduino_port
    arduino.open()
    print(arduino.is_open)
    print(arduino.readline())
    return arduino


"""
if face was found return the direction of the lower motor 
else return none
"""


def calibrateLower():    
    detect = False
    count = 0
    # looping until face is found or rotated 90 degrees to the right
    while(count < calibrateLowerTotalStepsCount):
        moveMotor(Motors.LOWER.value, calibrateLowerStepSize,
                  Direction.POSITIVE.value)
        count += 1
        result = radar.get_median_distance(1) 
        if result != -1:
            detect = True
                 
        if(detect):
            return Direction.POSITIVE.value  # return that a face is found when rotating right
    moveMotor(Motors.LOWER.value, calibrateLowerStepSize *
              calibrateLowerTotalStepsCount, Direction.NEGATIVE.value)
    count = 0
    # looping until face is found or rotated 90 degrees to the left
    while(not detect and count > -1 * calibrateLowerTotalStepsCount):
        moveMotor(Motors.LOWER.value, calibrateLowerStepSize,
                  Direction.NEGATIVE.value)
        count -= 1
        result = radar.get_median_distance(1) 
        if result != -1:
            detect = True
        if(detect):
            return Direction.NEGATIVE.value  # return that a face is found when rotating left
    return None


"""
Moves motor in arduino
motor --> can be either 'l' for lower motor 
          or  'u' for upper motor
stepSize --> (integer) number of steps that the motor will move (step = 0.45 angle)
direction -->  either -1 or 1 
"""


def moveMotor(motor: Motors, stepSize, direction: Direction):
    txt = motor + str(direction * stepSize) + "$"
    arduino.write(bytes(txt, 'utf-8'))
    time.sleep(.1)
    arduino.readline()


def scanFace(lowerDirection):
    global global_distance
    upperDirection = False
    
    moveU = True
    moveL = True
    uCounter = 0
    lCounter = 0
    count = 0
    dResult =[]
    uResult = []
    lResult = []
    while(moveL):

        while(moveU):
            if(upperDirection):
                moveMotor(Motors.UPPER.value, scanningUpperStepSize,
                          Direction.POSITIVE.value)
                uCounter += 1
                count+=1
            else:
                moveMotor(Motors.UPPER.value, scanningUpperStepSize,
                          Direction.NEGATIVE.value)
                uCounter -= 1
                count+=1
            
            distance = global_distance
            print("######################################")
            print("upperMoter.distance = ",distance)
            print("upperMoter.count = ",count)
            print("upperMoter.uCounter = ",uCounter)
            print("upperMoter.lCounter = ",lCounter)
            print("######################################")
            if (distance != -1):
                dResult.append(distance)
                uResult.append((uCounter * 0.45*np.pi)/180)
                lResult.append((lCounter * 0.45*np.pi)/180)
            # if distance >= min_distance and distance <= max_distance:
            #     detect = True
            #     print(distance)
            #     moveU = True
            # else:
            #     moveU = False
            print("upperMoter.count = ",count)
            if count == maxStepsOfUpper:
                moveU = False
                count = 0
        moveU = True
        upperDirection = not upperDirection
        # if(upperDirection):
        #     moveMotor(Motors.UPPER.value, scanningUpperStepSize,
        #               Direction.POSITIVE.value)
        #     uCounter += 1
        # else:
        #     moveMotor(Motors.UPPER.value, scanningUpperStepSize,
        #               Direction.NEGATIVE.value)
        #     uCounter -= 1

        moveMotor(Motors.LOWER.value, scanningLowerStepSize, lowerDirection)
        lCounter += 1
        distance = global_distance
        if (distance != -1):
            dResult.append(distance)
            uResult.append((uCounter * 0.45*np.pi)/180)
            lResult.append((lCounter * 0.45*np.pi)/180)
        if lCounter == maxStepsOfLower:
            moveL = False
        # if distance >= min_distance and distance <= max_distance:
        #     detect = True
        #     print(distance)
        #     moveL = True
        # else:
        #     moveL = False
        #     print("noooooooooooooooooooo")
        #     print(distance)
        #     print("noooooooooooooooooooo")

    return dResult,uResult,lResult

def move_with_keyboard ():
    val = ""
    while val != "e":
        val = input("Enter your value: ") 
        if (val == "a"):
            moveMotor(Motors.LOWER.value, scanningLowerStepSize, Direction.POSITIVE.value)
        elif (val == "d"):
            moveMotor(Motors.LOWER.value, scanningLowerStepSize, Direction.NEGATIVE.value)
        elif (val == "w"):
            moveMotor(Motors.UPPER.value, scanningUpperStepSize, Direction.NEGATIVE.value)
        elif (val == "s"):
            moveMotor(Motors.UPPER.value, scanningUpperStepSize, Direction.POSITIVE.value)
if __name__ == "__main__":
    

    arduino = set_up()
    # setting upp arduino ports
    t1 = Thread(target=get_readings_thread,daemon=False)
    t1.start()
    #move_with_keyboard ()
   

    # moves the sensor in lower direction (XY plane) until the face is found
    
    # lowerDirection = calibrateLower()
    lowerDirection = 1
    dist = []
    uAngel =[]
    lAngel = []
    if(lowerDirection is None):
        moveMotor(Motors.LOWER.value, maxStepsOfLower,
                  Direction.POSITIVE.value)
    else:
        dist,uAngel,lAngel = scanFace(lowerDirection)
    
        x , y , z = np.array(dist)*np.cos(uAngel)*np.sin(lAngel) , np.array(dist)*np.cos(uAngel)*np.cos(lAngel) , np.array(dist)*np.sin(uAngel)
        print(x)
        print(y)
        print(z)

        my_sample_x = np.array(x)
        my_sample_y = np.array(y)
        my_sample_z = np.array(z)
        
        cat_g = ['setosa']
        sample_cat = [cat_g[np.random.randint(0,1)] for i in range (len(my_sample_z))]

        df = pd.DataFrame(my_sample_x,columns=['sepal_length'])
        df['sepal_width'] = my_sample_y
        df['petal_width'] = my_sample_z
        df['species'] = sample_cat
        df.head()
        fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
                color='species',range_x = [-500,500],range_y = [-500,500],range_z=[-500,500])
        fig.show()

istance 388
with magnitude =  -73
1024
there is a peak at distance 392
with magnitude =  -71
global distance =  -1
######################################
-1
######################################
12
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
######################################
-1
######################################
13
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
######################################
-1
######################################
14
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
1024
there is a peak at distance 384
with magnitude =  -75
1024
there is a peak at distance 382
with magnitude =  -76
1024
there is a peak at distance 596
with magnitude =  -75
global distance =  -1
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
######################################
-1
######################################
15
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
######################################
-1
######################################
16
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
1024
there is a peak

In [2]:
my_sample_x = np.array(x)
my_sample_y = np.array(y)
my_sample_z = np.array(z)

In [4]:
my_sample_x

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
1024
there is a peak at distance 334
with magnitude =  -50
1024
there is a peak at distance 334
with magnitude =  -51
1024
there is a peak at distance 334
with magnitude =  -51
global distance =  -1


array([  0.        ,   0.        ,   0.        , ..., 241.42576438,
       236.60675221, 234.90854456])

In [5]:
my_sample_y

array([493.75624082, 493.6191439 , 493.45159823, ..., 245.24817113,
       240.35286128, 238.62776653])

In [6]:
my_sample_z

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024


array([-15.51691498, -19.39434898, -23.27058665, ..., 308.23843831,
       306.89370616, 309.53313996])

In [7]:
len(my_sample_z)

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
1024
there is a peak at distance 338
with magnitude =  -50
1024
there is a peak at distance 334
with magnitude =  -51
1024
there is a peak at distance 334
with magnitude =  -51
global distance =  -1
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
1024
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
557


6314

In [10]:
file = open("z_array.txt",'w')
file.writelines([str(z) for z in my_sample_z])

In [11]:
file = open("x_array.txt",'w')
file.writelines([str(z) for z in my_sample_x])

In [12]:
file = open("y_array.txt",'w')
file.writelines([str(z) for z in my_sample_y])

In [35]:
my_sample_z_2 = my_sample_z*10
my_sample_x_2 = my_sample_x*10
my_sample_y_2 = my_sample_y*10

In [36]:
cat_g = ['setosa']
sample_cat = [cat_g[np.random.randint(0,1)] for i in range (len(my_sample_z))]

df = pd.DataFrame(my_sample_x_2,columns=['sepal_length'])
df['sepal_width'] = my_sample_y_2
df['petal_width'] = my_sample_z_2
df['species'] = sample_cat
df.head()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
    color='species',range_x = [-5000,5000],range_y = [-5000,5000],range_z=[-5000,5000], size_max=1)
fig.show()